In [1]:
import numpy as np
from gurobipy import *
import random
import matplotlib.pyplot as plot
import csv

In [2]:
data=[]

data = np.loadtxt('C:/Users/user/test/data/processed.cleveland.data',delimiter=',',dtype=str)

In [3]:
L=len(data)
l=len(data[0])-1
cn = len(np.unique(list(zip(*data))[l]))
x=[]
for i in range(L):
    x.append(i)
    for j in range(l):
        if data[i,j]=='?':
            x.remove(i)
data = data[x]
data = data.astype('float64')
ci=2 # class identifier
smallnum=1.e-3
bignum=1.e+3

L=len(data)
l = len(data[0])-1
#data1 = np.c_[data,np.zeros(L)]

np.random.shuffle(data)

In [4]:
train=data[:int((len(data)+1)*.80)]
test=data[int(len(data)*.80+1):]

In [5]:
len(train)

238

In [6]:
len(data)

297

In [7]:
Atrain=list()
Btrain=list()
Atest=list()
Btest=list()

In [8]:
for i in range(len(train)):
    if train[i,-1] == 0:
        Atrain.append(train[i])
    else :
        Btrain.append(train[i])

Atrain=np.array(Atrain)
Btrain=np.array(Btrain)

lA = len(Atrain)
lB = len(Btrain)

In [9]:
m=Model()
w=[]
y=[]
z=[]

Academic license - for non-commercial use only


In [10]:
r=m.addVar(vtype=GRB.CONTINUOUS, name='r')

for i in range(lA):
    y.append(m.addVar(vtype=GRB.CONTINUOUS, name="y[{}]".format(i),lb=0))
    
for i in range(lB):
    z.append(m.addVar(vtype=GRB.CONTINUOUS, name="z[{}]".format(i),lb=0))
    
for i in range(l):
    w.append(m.addVar(vtype=GRB.CONTINUOUS, name="w[{}]".format(i),lb=-100,ub=100))

m.update()

In [11]:
Y=sum(y[i] for i in range(lA))
Z=sum(z[i] for i in range(lB))

m.setObjective(Y/lA + Z/lB , GRB.MINIMIZE)
m.update()

In [12]:
for i in range(lA):
    m.addConstr(sum(Atrain[i,j]*w[j] for j in range(l))-r+y[i] >=smallnum, name = "cty[{}]".format(i))
m.update()
for i in range(lB):
    m.addConstr(sum(Btrain[i,j]*w[j] for j in range(l))-r-z[i] <=-smallnum, name = "ctz[{}]".format(i))
m.update()

m.optimize()

Optimize a model with 238 rows, 252 columns and 2791 nonzeros
Coefficient statistics:
  Matrix range     [1e-01, 6e+02]
  Objective range  [8e-03, 9e-03]
  Bounds range     [1e+02, 1e+02]
  RHS range        [1e-03, 1e-03]
Presolve time: 0.01s
Presolved: 238 rows, 252 columns, 2791 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.419444e+05   0.000000e+00      0s
     188    7.3461759e-04   0.000000e+00   0.000000e+00      0s

Solved in 188 iterations and 0.02 seconds
Optimal objective  7.346175862e-04


In [13]:
for i in range(len(test)):
    if test[i,-1]==0:
        Atest.append(test[i])
    else : 
        Btest.append(test[i])

Atest = np.array(Atest)
Btest = np.array(Btest)

fail = 0
for i in range(len(Atest)):
    if(sum(Atest[i,j]*w[j].x for j in range(l))<r.x):
        fail +=1

for i in range(len(Btest)):
    if(sum(Btest[i,j]*w[j].x for j in range(l))>r.x):
        fail+=1

In [14]:
success_rate = (len(test)-fail)*100/len(test)
print(success_rate)

84.7457627118644


In [15]:
Atrain=list()
Btrain=list()
Atest=list()
Btest=list()

for i in range(len(train)):
    if train[i,-1] == 1:
        Atrain.append(train[i])
    else :
        Btrain.append(train[i])

Atrain=np.array(Atrain)
Btrain=np.array(Btrain)

lA = len(Atrain)
lB = len(Btrain)

m=Model()
w=[]
y=[]
z=[]

r=m.addVar(vtype=GRB.CONTINUOUS, name='r')

for i in range(lA):
    y.append(m.addVar(vtype=GRB.CONTINUOUS, name="y[{}]".format(i),lb=0))
    
for i in range(lB):
    z.append(m.addVar(vtype=GRB.CONTINUOUS, name="z[{}]".format(i),lb=0))
    
for i in range(l):
    w.append(m.addVar(vtype=GRB.CONTINUOUS, name="w[{}]".format(i),lb=-100,ub=100))

m.update()

Y=sum(y[i] for i in range(lA))
Z=sum(z[i] for i in range(lB))

m.setObjective(Y/lA + Z/lB , GRB.MINIMIZE)
m.update()

for i in range(lA):
    m.addConstr(sum(Atrain[i,j]*w[j] for j in range(l))-r+y[i] >=smallnum, name = "cty[{}]".format(i))
m.update()
for i in range(lB):
    m.addConstr(sum(Btrain[i,j]*w[j] for j in range(l))-r-z[i] <=-smallnum, name = "ctz[{}]".format(i))
m.update()

m.optimize()

for i in range(len(test)):
    if test[i,-1]==1:
        Atest.append(test[i])
    else : 
        Btest.append(test[i])

Atest = np.array(Atest)
Btest = np.array(Btest)

fail = 0
for i in range(len(Atest)):
    if(sum(Atest[i,j]*w[j].x for j in range(l))<r.x):
        fail +=1

for i in range(len(Btest)):
    if(sum(Btest[i,j]*w[j].x for j in range(l))>r.x):
        fail+=1
        
success_rate = (len(test)-fail)*100/len(test)
print(success_rate)

Optimize a model with 238 rows, 252 columns and 2791 nonzeros
Coefficient statistics:
  Matrix range     [1e-01, 6e+02]
  Objective range  [5e-03, 2e-02]
  Bounds range     [1e+02, 1e+02]
  RHS range        [1e-03, 1e-03]
Presolve time: 0.01s
Presolved: 238 rows, 252 columns, 2791 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   8.242688e+04   0.000000e+00      0s
     277    1.4664307e-03   0.000000e+00   0.000000e+00      0s

Solved in 277 iterations and 0.03 seconds
Optimal objective  1.466430689e-03
62.71186440677966


In [16]:
Atrain=list()
Btrain=list()
Atest=list()
Btest=list()

for i in range(len(train)):
    if train[i,-1] == 2:
        Atrain.append(train[i])
    else :
        Btrain.append(train[i])

Atrain=np.array(Atrain)
Btrain=np.array(Btrain)

lA = len(Atrain)
lB = len(Btrain)

m=Model()
w=[]
y=[]
z=[]

r=m.addVar(vtype=GRB.CONTINUOUS, name='r')

for i in range(lA):
    y.append(m.addVar(vtype=GRB.CONTINUOUS, name="y[{}]".format(i),lb=0))
    
for i in range(lB):
    z.append(m.addVar(vtype=GRB.CONTINUOUS, name="z[{}]".format(i),lb=0))
    
for i in range(l):
    w.append(m.addVar(vtype=GRB.CONTINUOUS, name="w[{}]".format(i),lb=-100,ub=100))

m.update()

Y=sum(y[i] for i in range(lA))
Z=sum(z[i] for i in range(lB))

m.setObjective(Y/lA + Z/lB , GRB.MINIMIZE)
m.update()

for i in range(lA):
    m.addConstr(sum(Atrain[i,j]*w[j] for j in range(l))-r+y[i] >=smallnum, name = "cty[{}]".format(i))
m.update()
for i in range(lB):
    m.addConstr(sum(Btrain[i,j]*w[j] for j in range(l))-r-z[i] <=-smallnum, name = "ctz[{}]".format(i))
m.update()

m.optimize()

for i in range(len(test)):
    if test[i,-1]==2:
        Atest.append(test[i])
    else : 
        Btest.append(test[i])

Atest = np.array(Atest)
Btest = np.array(Btest)

fail = 0
for i in range(len(Atest)):
    if(sum(Atest[i,j]*w[j].x for j in range(l))<r.x):
        fail +=1

for i in range(len(Btest)):
    if(sum(Btest[i,j]*w[j].x for j in range(l))>r.x):
        fail+=1
        
success_rate = (len(test)-fail)*100/len(test)
print(success_rate)

Optimize a model with 238 rows, 252 columns and 2791 nonzeros
Coefficient statistics:
  Matrix range     [1e-01, 6e+02]
  Objective range  [5e-03, 3e-02]
  Bounds range     [1e+02, 1e+02]
  RHS range        [1e-03, 1e-03]
Presolve time: 0.01s
Presolved: 238 rows, 252 columns, 2791 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.500844e+04   0.000000e+00      0s
     196    8.9700162e-04   0.000000e+00   0.000000e+00      0s

Solved in 196 iterations and 0.03 seconds
Optimal objective  8.970016167e-04
66.10169491525424


In [17]:
Atrain=list()
Btrain=list()
Atest=list()
Btest=list()

for i in range(len(train)):
    if train[i,-1] == 3:
        Atrain.append(train[i])
    else :
        Btrain.append(train[i])

Atrain=np.array(Atrain)
Btrain=np.array(Btrain)

lA = len(Atrain)
lB = len(Btrain)

m=Model()
w=[]
y=[]
z=[]

r=m.addVar(vtype=GRB.CONTINUOUS, name='r')

for i in range(lA):
    y.append(m.addVar(vtype=GRB.CONTINUOUS, name="y[{}]".format(i),lb=0))
    
for i in range(lB):
    z.append(m.addVar(vtype=GRB.CONTINUOUS, name="z[{}]".format(i),lb=0))
    
for i in range(l):
    w.append(m.addVar(vtype=GRB.CONTINUOUS, name="w[{}]".format(i),lb=-100,ub=100))

m.update()

Y=sum(y[i] for i in range(lA))
Z=sum(z[i] for i in range(lB))

m.setObjective(Y/lA + Z/lB , GRB.MINIMIZE)
m.update()

for i in range(lA):
    m.addConstr(sum(Atrain[i,j]*w[j] for j in range(l))-r+y[i] >=smallnum, name = "cty[{}]".format(i))
m.update()
for i in range(lB):
    m.addConstr(sum(Btrain[i,j]*w[j] for j in range(l))-r-z[i] <=-smallnum, name = "ctz[{}]".format(i))
m.update()

m.optimize()

for i in range(len(test)):
    if test[i,-1]==3:
        Atest.append(test[i])
    else : 
        Btest.append(test[i])

Atest = np.array(Atest)
Btest = np.array(Btest)

fail = 0
for i in range(len(Atest)):
    if(sum(Atest[i,j]*w[j].x for j in range(l))<r.x):
        fail +=1

for i in range(len(Btest)):
    if(sum(Btest[i,j]*w[j].x for j in range(l))>r.x):
        fail+=1
        
success_rate = (len(test)-fail)*100/len(test)
print(success_rate)

Optimize a model with 238 rows, 252 columns and 2791 nonzeros
Coefficient statistics:
  Matrix range     [1e-01, 6e+02]
  Objective range  [5e-03, 4e-02]
  Bounds range     [1e+02, 1e+02]
  RHS range        [1e-03, 1e-03]
Presolve time: 0.01s
Presolved: 238 rows, 252 columns, 2791 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.199281e+04   0.000000e+00      0s
     184    7.8968086e-04   0.000000e+00   0.000000e+00      0s

Solved in 184 iterations and 0.03 seconds
Optimal objective  7.896808634e-04
66.10169491525424


In [18]:
Atrain=list()
Btrain=list()
Atest=list()
Btest=list()

for i in range(len(train)):
    if train[i,-1] == 4:
        Atrain.append(train[i])
    else :
        Btrain.append(train[i])

Atrain=np.array(Atrain)
Btrain=np.array(Btrain)

lA = len(Atrain)
lB = len(Btrain)

m=Model()
w=[]
y=[]
z=[]

r=m.addVar(vtype=GRB.CONTINUOUS, name='r')

for i in range(lA):
    y.append(m.addVar(vtype=GRB.CONTINUOUS, name="y[{}]".format(i),lb=0))
    
for i in range(lB):
    z.append(m.addVar(vtype=GRB.CONTINUOUS, name="z[{}]".format(i),lb=0))
    
for i in range(l):
    w.append(m.addVar(vtype=GRB.CONTINUOUS, name="w[{}]".format(i),lb=-100,ub=100))

m.update()

Y=sum(y[i] for i in range(lA))
Z=sum(z[i] for i in range(lB))

m.setObjective(Y/lA + Z/lB , GRB.MINIMIZE)
m.update()

for i in range(lA):
    m.addConstr(sum(Atrain[i,j]*w[j] for j in range(l))-r+y[i] >=smallnum, name = "cty[{}]".format(i))
m.update()
for i in range(lB):
    m.addConstr(sum(Btrain[i,j]*w[j] for j in range(l))-r-z[i] <=-smallnum, name = "ctz[{}]".format(i))
m.update()

m.optimize()

for i in range(len(test)):
    if test[i,-1]==4:
        Atest.append(test[i])
    else : 
        Btest.append(test[i])

Atest = np.array(Atest)
Btest = np.array(Btest)

fail = 0
for i in range(len(Atest)):
    if(sum(Atest[i,j]*w[j].x for j in range(l))<r.x):
        fail +=1

for i in range(len(Btest)):
    if(sum(Btest[i,j]*w[j].x for j in range(l))>r.x):
        fail+=1
        
success_rate = (len(test)-fail)*100/len(test)
print(success_rate)

Optimize a model with 238 rows, 252 columns and 2791 nonzeros
Coefficient statistics:
  Matrix range     [1e-01, 6e+02]
  Objective range  [4e-03, 2e-01]
  Bounds range     [1e+02, 1e+02]
  RHS range        [1e-03, 1e-03]
Presolve removed 0 rows and 1 columns
Presolve time: 0.02s
Presolved: 238 rows, 251 columns, 2758 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.151156e+04   0.000000e+00      0s
Extra 2 simplex iterations after uncrush
     138    5.1329655e-04   0.000000e+00   0.000000e+00      0s

Solved in 138 iterations and 0.04 seconds
Optimal objective  5.132965511e-04
88.13559322033899
